In [1]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

e:\Generative-AI\genai-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(
        page_content=(
            """The Grand Canyon is one of the most visited natural wonders in the world.
            Photosynthesis is the process by which green plants convert sunlight into energy.
            Millions of tourists travel to see it every year. The rocks date back millions of years."""
        ),
        metadata={"source": "Doc1"}
    ),

    Document(
        page_content=(
            """In medieval Europe, castles were built primarily for defense.
            The chlorophyll in plant cells captures sunlight during photosynthesis.
            Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
        ),
        metadata={"source": "Doc2"}
    ),

    Document(
        page_content=(
            """Basketball was invented by Dr. James Naismith in the late 19th century.
            It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
        ),
        metadata={"source": "Doc3"}
    ),

    Document(
        page_content=(
            """The history of cinema began in the late 1800s. Silent films were the earliest form.
            Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
            Modern filmmaking involves complex CGT and sound design."""
        ),
        metadata={"source": "Doc4"}
    )
]


In [3]:
# create embedding model
embedding_model = OllamaEmbeddings(
    model="all-minilm"
)

In [4]:
# create FAISS vector store
faiss_vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [5]:
# create base retriever, and llm and define the contextualcompressor retriever using them
base_retriever = faiss_vector_store.as_retriever(search_kwargs={"k":5})

llm = ChatGroq(model="llama-3.3-70b-versatile")
compressor = LLMChainExtractor.from_llm(llm)

# create the compressor retriever now
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [6]:
# now get an user query and send it to the retriever and fetch the top k results
query = "What is photosynthesis?"

compressed_results = compression_retriever.invoke(query)

for i,doc in enumerate(compressed_results):
    print(f"Result {i+1}:")
    print(doc.page_content)

Result 1:
Photosynthesis is the process by which green plants convert sunlight into energy.
Result 2:
The chlorophyll in plant cells captures sunlight during photosynthesis.
Result 3:
Photosynthesis does not occur in animal cells.


Each document is a paragraph long, but the relevant search results we get from the query is just a sentence. This is because the documents has been compressed by the LLMChainExtractor.